In [60]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('olist').getOrCreate()

25/05/15 06:09:01 INFO SparkEnv: Registering MapOutputTracker
25/05/15 06:09:01 INFO SparkEnv: Registering BlockManagerMaster
25/05/15 06:09:01 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
25/05/15 06:09:01 INFO SparkEnv: Registering OutputCommitCoordinator


In [61]:
hdfs_path = '/data/olist/'

In [62]:
customers_df = spark.read.csv(hdfs_path + 'olist_customers_dataset.csv',header=True,inferSchema=True)
geolocation_df = spark.read.csv(hdfs_path + 'olist_geolocation_dataset.csv',header=True,inferSchema=True)
order_items_df = spark.read.csv(hdfs_path + 'olist_order_items_dataset.csv',header=True,inferSchema=True)
payments_df = spark.read.csv(hdfs_path + 'olist_order_payments_dataset.csv',header=True,inferSchema=True)
reviews_df = spark.read.csv(hdfs_path + 'olist_order_reviews_dataset.csv',header=True,inferSchema=True)
orders_df = spark.read.csv(hdfs_path + 'olist_orders_dataset.csv',header=True,inferSchema=True)
products_df = spark.read.csv(hdfs_path + 'olist_products_dataset.csv',header=True,inferSchema=True)
sellers_df = spark.read.csv(hdfs_path + 'olist_sellers_dataset.csv',header=True,inferSchema=True)
category_translation_df = spark.read.csv(hdfs_path + 'product_category_name_translation.csv',header=True,inferSchema=True)

In [63]:
from pyspark.sql.functions import *

# Cache Frequently used Data for Better Performance

In [65]:
orders_df.cache()
customers_df.cache()
order_items_df.cache()

DataFrame[order_id: string, order_item_id: int, product_id: string, seller_id: string, shipping_limit_date: timestamp, price: double, freight_value: double]

In [20]:
full_order_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [23]:
# Total Revenues Per Seller

seller_revenue_df = full_order_df.groupBy('seller_id').agg(format_number(sum('price'), 2).alias('total_revenue'))

In [15]:
seller_revenue_df.show(5)

+--------------------+-------------+
|           seller_id|total_revenue|
+--------------------+-------------+
|8e6cc767478edae94...| 1,145,757.40|
|4d600e08ecbe08258...|   436,434.23|
|9b1050e85becf3ae9...|    19,667.34|
|cb5ff1b9715e99589...|    13,235.00|
|038b75b729c8a9a04...|    17,979.50|
+--------------------+-------------+
only showing top 5 rows



In [22]:
# Total Orders Per Customer
Orders_per_customer = full_order_df.groupBy('customer_id')\
.agg(count('order_id').alias('total_orders'))\
     .orderBy('total_orders',ascending = False)
     
Orders_per_customer.show()

+--------------------+------------+
|         customer_id|total_orders|
+--------------------+------------+
|351e40989da90e704...|       11427|
|50920f8cd0681fd86...|       10752|
|9b43e2a62de9bab3a...|        8556|
|270c23a11d024a44c...|        8001|
|5c87184371002d49e...|        6876|
|d3e82ccec3cb5f956...|        6876|
|d5f2b3f597c7ccafb...|        6706|
|c2f18647725395af4...|        6612|
|24e7dc2ff8c071263...|        6597|
|7bb57d182bdc11653...|        6258|
|63b964e79dee32a35...|        6072|
|d22f25a9fadfb1abb...|        6072|
|1ff773612ab8934db...|        5820|
|13aa59158da63ba0e...|        5206|
|78fc46047c4a639e8...|        5200|
|dd3f1762eb601f41c...|        4992|
|a193aa8d905b8e246...|        4896|
|9eb3d566e87289dcb...|        4872|
|2ba91e12e5e4c9f56...|        4752|
|55e7cfd6e28d2fbfb...|        4728|
+--------------------+------------+
only showing top 20 rows



In [24]:
# Avg review score per seller
full_order_df = full_order_df.withColumn('review_score',col('review_score').cast('int'))
avg_review_per_seller = full_order_df.groupBy('seller_id')\
.agg(avg('review_score').alias('avg_review_score'))\
.orderBy('avg_review_score',ascending = False)

avg_review_per_seller.show()

+--------------------+----------------+
|           seller_id|avg_review_score|
+--------------------+----------------+
|58c851d1a3c7cd3da...|             5.0|
|aa8af66c623d7d544...|             5.0|
|0f519b0d2e5eb2227...|             5.0|
|31e60bf8d103ce479...|             5.0|
|64c9a1db4e73e19aa...|             5.0|
|2b2fed75b8e5ea3a0...|             5.0|
|33ab10be054370c25...|             5.0|
|9d213f303afae4983...|             5.0|
|05a48cc8859962767...|             5.0|
|94d76e96eedd97625...|             5.0|
|e94b64dc6979b302a...|             5.0|
|4aba6a02a788d3ec8...|             5.0|
|fd312b6bf05efac6c...|             5.0|
|a61cc04793308395a...|             5.0|
|1a8e2d9c38b84a970...|             5.0|
|edb58a1390adf2738...|             5.0|
|c04d70d515d15a7e3...|             5.0|
|5f57db27027655e6c...|             5.0|
|c74f14c1e26cf1bd5...|             5.0|
|2c538755f1ca9540a...|             5.0|
+--------------------+----------------+
only showing top 20 rows



In [25]:
# Most sold products (top 10)

delivered_orders = full_order_df.filter(col("order_status") == "delivered")

top_products = delivered_orders.groupBy('product_id')\
.agg(count('order_id').alias('total_sold'))\
     .orderBy('total_sold',ascending = False).limit(10)
     
top_products.show()

+--------------------+----------+
|          product_id|total_sold|
+--------------------+----------+
|aca2eb7d00ea1a7b8...|     85185|
|422879e10f4668299...|     81110|
|99a4788cb24856965...|     77751|
|389d119b48cf3043d...|     59245|
|368c6c730842d7801...|     58358|
|d1c427060a0f73f6b...|     57883|
|53759a2ecddad2bb8...|     52654|
|53b36df67ebb7c415...|     52002|
|154e7e31ebfa09220...|     41770|
|3dd2a17168ec895c7...|     40642|
+--------------------+----------+



In [26]:
top_products_category = delivered_orders.groupBy('product_category_name')\
.agg(count('order_id').alias('total_sold'))\
     .orderBy('total_sold',ascending = False).limit(10)
     
top_products_category.show()

+---------------------+----------+
|product_category_name|total_sold|
+---------------------+----------+
|      cama_mesa_banho|   1872019|
|         beleza_saude|   1421143|
|     moveis_decoracao|   1396012|
|        esporte_lazer|   1369521|
| informatica_acess...|   1235028|
| utilidades_domest...|   1171814|
|   relogios_presentes|    871948|
|   ferramentas_jardim|    683645|
|           brinquedos|    656063|
|            telefonia|    615745|
+---------------------+----------+



In [27]:
# Top customers by spending

top_customers_spending = full_order_df.groupBy('customer_id')\
.agg(sum('payment_value').alias('total_spending'))\
     .orderBy('total_spending',ascending = False)\
.withColumn("total_spending", format_number("total_spending", 2))

top_customers_spending.show()

+--------------------+--------------+
|         customer_id|total_spending|
+--------------------+--------------+
|1ff773612ab8934db...| 17,568,252.00|
|05455dfa7cd02f13d...| 13,282,083.36|
|ec5b2ba62e5743423...| 10,388,528.64|
|0c792d32a3251b4f6...|  8,254,681.60|
|78fc46047c4a639e8...|  7,488,520.00|
|1617b1357756262bf...|  7,433,259.52|
|1dbc055ccab23ed89...|  7,216,273.40|
|d5f2b3f597c7ccafb...|  6,800,018.12|
|dd3f1762eb601f41c...|  6,746,388.48|
|10de381f8a8d23fff...|  5,184,499.50|
|30bb84b541c96af98...|  4,740,404.55|
|d72181923840c8895...|  4,513,322.70|
|e7d6802668de6e74d...|  4,000,041.40|
|cb87122c4871e2027...|  3,957,404.00|
|a972623b3481cbfd9...|  3,716,577.36|
|df55c14d1476a9a34...|  3,678,102.62|
|f7622098214b4634b...|  3,657,923.52|
|cc803a2c412833101...|  3,387,471.00|
|f4d64a735d1f90f13...|  3,224,595.42|
|6361b9f3b85d41860...|  3,207,389.94|
+--------------------+--------------+
only showing top 20 rows



# Optimized Joins For Data Integration

In [66]:
order_items_joined_df= orders_df.join(order_items_df,'order_id','inner')

In [67]:
order_items_products_df = order_items_joined_df.join(products_df,'product_id','inner')

In [68]:
order_items_products_sellers_df = order_items_products_df.join(broadcast(sellers_df),'seller_id','inner')

In [69]:
full_order_df = order_items_products_sellers_df.join(customers_df,'customer_id','inner')

In [70]:
# Geolocation Data

full_order_df = full_order_df.join(broadcast(geolocation_df),full_order_df.customer_zip_code_prefix==geolocation_df.geolocation_zip_code_prefix,'left')

In [71]:
full_order_df = full_order_df.join(broadcast(reviews_df),'order_id','left')

In [72]:
full_order_df = full_order_df.join(payments_df,'order_id','left')

In [73]:
full_order_df.cache()

DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

# Window Function and Ranking


In [36]:
from pyspark.sql.window import Window

In [37]:


window_spec = Window.partitionBy('seller_id').orderBy(desc('price'))


In [38]:
# Rank top selling products per seller

top_seller_products_df = full_order_df.withColumn('rank',rank().over(window_spec)).filter(col('rank')<=5)

top_seller_products_df.select('seller_id','price','rank').show()

+--------------------+-----+----+
|           seller_id|price|rank|
+--------------------+-----+----+
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
+--------------------+-----+----+
only showing top 20 rows



In [39]:
# Dense rank for sellers based on revenue
seller_revenue_df = full_order_df.groupBy('seller_id')\
.agg(sum('price').alias('total_revenue'))\
.withColumn("total_revenue", format_number("total_revenue", 2))


In [40]:
window_spec = Window.orderBy(col('total_revenue').desc())

In [41]:
seller_ranked_df = seller_revenue_df.withColumn('revenue_rank',dense_rank().over(window_spec))

In [27]:
seller_ranked_df.orderBy('revenue_rank').show()

25/05/15 04:35:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/15 04:35:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/15 04:35:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/15 04:35:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/15 04:35:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/15 04:35:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/15 0

+--------------------+-------------+------------+
|           seller_id|total_revenue|revenue_rank|
+--------------------+-------------+------------+
|ebd2d60905fb58271...|   999,871.80|           1|
|ea67327e24487bdfa...|   999,241.84|           2|
|21e83881401b92b49...|   999,240.42|           3|
|a420f60ff1aa9acc8...|   999,172.50|           4|
|bba3293d6f5d5af2f...|   998,169.18|           5|
|516e7738bd8f735ac...|   997,490.93|           6|
|ff1fb4c404b2efe68...|   996,410.00|           7|
|da6a60cc8cc724fe5...|    99,947.55|           8|
|c4fabeea6e569d42d...|    99,892.90|           9|
|c6218512d16fcac0a...|    99,831.00|          10|
|4ce8b55b4bfa46591...|    99,441.00|          11|
|9dc79dd988ac42f4d...|    99,352.60|          12|
|6f83d9e4da896daae...|    99,320.65|          13|
|eb3071b9f0b0ddaa2...|   986,530.00|          14|
|fde0cc9ea29c8ccfc...|   984,684.00|          15|
|b87a7ea0c30fad2dd...|    98,753.70|          16|
|b81e3a6c866a76497...|    98,574.24|          17|


# Advance Aggregation and Enrichment

In [77]:
# Total Revenue & Average Order Value (AOV) per customer

customer_spending_df = full_order_df.groupBy('customer_id')\
.agg(count('order_id').alias('total_orders'),
     sum('price').alias('total_spend'),
     round(avg('price'),2).alias('AOV'))\
     .orderBy(desc('total_spend'))
     
customer_spending_df.show()


+--------------------+------------+------------------+-------+
|         customer_id|total_orders|       total_spend|    AOV|
+--------------------+------------+------------------+-------+
|d3e82ccec3cb5f956...|        6876|         6662844.0|  969.0|
|df55c14d1476a9a34...|         743|         3565657.0| 4799.0|
|fe5113a38e3575c04...|        2292|         3293604.0| 1437.0|
|ec5b2ba62e5743423...|        1428|         2556120.0| 1790.0|
|63b964e79dee32a35...|        6072|         2501664.0|  412.0|
|46bb3c0b1a65c8399...|         748|         2336752.0| 3124.0|
|05455dfa7cd02f13d...|        2184| 2160194.400000087|  989.1|
|3690e975641f01bd0...|         802|         2124498.0| 2649.0|
|349509b216bd5ec11...|         743|         1923627.0| 2589.0|
|695476b5848d64ba0...|         687|1820543.1299999943|2649.99|
|73236a0796f53d60d...|         832|         1755520.0| 2110.0|
|cc803a2c412833101...|         762|         1676400.0| 2200.0|
|1ff773612ab8934db...|        5820|1658641.7999999512| 

In [43]:
# Seller Performance Metrics (Revenue, Average Review, Order Count)

seller_performance_df = full_order_df.groupBy('seller_id')\
.agg(
    (sum('price')).alias('total_revenue'),
    round(avg('review_score'),2).alias('avg_review_score'),
    count('order_id').alias('total_orders'),
    round(stddev('price'),2).alias('price_variability')
)\
.withColumn('total_revenue',format_number('total_revenue',2))\
.orderBy(col('total_revenue').desc())

seller_performance_df.show()

+--------------------+-------------+----------------+------------+-----------------+
|           seller_id|total_revenue|avg_review_score|total_orders|price_variability|
+--------------------+-------------+----------------+------------+-----------------+
|ebd2d60905fb58271...|   999,871.80|            4.34|       10677|            59.42|
|ea67327e24487bdfa...|   999,241.84|            3.82|        6193|            88.83|
|21e83881401b92b49...|   999,240.42|            4.32|        4755|            93.25|
|a420f60ff1aa9acc8...|   999,172.50|            3.44|       10602|           123.89|
|bba3293d6f5d5af2f...|   998,169.18|            4.71|        1324|           105.79|
|516e7738bd8f735ac...|   997,490.93|            4.76|       13177|            49.95|
|ff1fb4c404b2efe68...|   996,410.00|            3.87|        7672|            96.02|
|da6a60cc8cc724fe5...|    99,947.55|            4.71|        1581|            27.12|
|c4fabeea6e569d42d...|    99,892.90|            3.59|        1300

In [44]:
# Product Popularity Metrics

product_metrics_df = full_order_df.groupBy('product_id')\
.agg(
    count('order_id').alias('total_sales'),
    sum('price').alias('total_revenue'),
    round(avg('price'),2).alias('avg_price'),
    round(stddev('price'),2).alias('price_volatility'),\
    collect_set('seller_id').alias('unique_sellers')
)\
.withColumn('total_revenue',format_number('total_revenue',2))\
.orderBy(desc('total_sales'))

In [31]:
product_metrics_df.show()

+--------------------+-----------+-------------+---------+----------------+--------------------+
|          product_id|total_sales|total_revenue|avg_price|price_volatility|      unique_sellers|
+--------------------+-----------+-------------+---------+----------------+--------------------+
|aca2eb7d00ea1a7b8...|      86740| 6,164,630.30|    71.07|            3.17|[955fee9216a65b61...|
|422879e10f4668299...|      81110| 4,442,791.51|    54.77|            4.46|[1f50f920176fa81d...|
|99a4788cb24856965...|      78775| 6,921,762.71|    87.87|            4.08|[4a3ca9315b744ce9...|
|389d119b48cf3043d...|      60248| 3,280,533.13|    54.45|            4.37|[1f50f920176fa81d...|
|d1c427060a0f73f6b...|      59274| 8,220,103.33|   138.68|           16.58|[a1043bafd471dff5...|
|368c6c730842d7801...|      58358| 3,181,698.90|    54.52|            4.59|[1f50f920176fa81d...|
|53759a2ecddad2bb8...|      52654| 2,893,017.50|    54.94|            4.52|[1f50f920176fa81d...|
|53b36df67ebb7c415...|      52

# Monthly Revenue and Order count Trend 

order_purchase_timestamp --> extract month
total_orders
total_revenue
avg_order_value
min_order_value
max_order_value

In [45]:
monthly_trend_df = full_order_df.withColumn('order_month',date_format('order_purchase_timestamp','yyyy-MM'))\
                                            .groupBy('order_month')\
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('total_revenue'),
    round(avg('price'),2).alias('avg_order_value'),
    round(min('price'),2).alias('min_order_value'),
    round(max('price'),2).alias('max_order_value')
)\
.withColumn('total_revenue',format_number('total_revenue',2))\
.orderBy(desc('order_month'))

monthly_trend_df.show()

+-----------+------------+--------------+---------------+---------------+---------------+
|order_month|total_orders| total_revenue|avg_order_value|min_order_value|max_order_value|
+-----------+------------+--------------+---------------+---------------+---------------+
|    2018-09|          33|      4,785.00|          145.0|          145.0|          145.0|
|    2018-08|     1112919|135,340,794.09|         121.61|            2.2|        4399.87|
|    2018-07|     1086810|138,919,305.66|         127.82|            3.0|         6729.0|
|    2018-06|     1123394|140,667,502.12|         125.22|            3.5|         4590.0|
|    2018-05|     1237049|156,856,056.59|          126.8|            3.9|         3930.0|
|    2018-04|     1265754|158,851,337.12|          125.5|           0.85|        3399.99|
|    2018-03|     1316199|154,576,832.52|         117.44|           4.99|        4099.99|
|    2018-02|     1236262|141,066,599.15|         114.11|           2.99|         3099.9|
|    2018-

In [34]:
full_order_df.select('order_status').distinct().show(truncate=False)


+------------+
|order_status|
+------------+
|delivered   |
|canceled    |
|shipped     |
|approved    |
|processing  |
|invoiced    |
|unavailable |
+------------+



In [74]:
# Order status flags

full_order_df = full_order_df.withColumn(
    'order_status_flag',
    when(col('order_status')=='delivered','completed')
    .when(col('order_status')== 'shipped','in-transit')
    .when(col('order_status').isin('approved','processsing','invoiced'),'in-Progress')
    .when(col('order_status').isin('canceled','unavialble'),'cancelled')
    .otherwise('other')
                                  )

In [75]:
full_order_df= full_order_df\
.withColumn('is_delivered',when(col('order_status_flag')=='completed',lit(1)).otherwise(lit(0)))\
.withColumn('is_cancelled',when(col('order_status_flag')=='cancelled',lit(1)).otherwise(lit(0)))

In [34]:
full_order_df.where(full_order_df['order_status_flag']=='cancelled').select('order_status','is_delivered','is_cancelled').show(10)

+------------+------------+------------+
|order_status|is_delivered|is_cancelled|
+------------+------------+------------+
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
+------------+------------+------------+
only showing top 10 rows



In [76]:
# Order Revenue Calculation

full_order_df = full_order_df.withColumn('order_revenue',col('price')+col('freight_value'))

In [40]:
full_order_df.select('price','freight_value','order_revenue').show()

+-----+-------------+-------------+
|price|freight_value|order_revenue|
+-----+-------------+-------------+
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
+-----+-------------+-------------+
only showing top 20 rows



In [41]:
customer_spending_df.show()

+--------------------+------------+------------------+-------+
|         customer_id|total_orders|       total_spend|    AOV|
+--------------------+------------+------------------+-------+
|d3e82ccec3cb5f956...|        6876|         6662844.0|  969.0|
|df55c14d1476a9a34...|         743|         3565657.0| 4799.0|
|fe5113a38e3575c04...|        2292|         3293604.0| 1437.0|
|ec5b2ba62e5743423...|        1428|         2556120.0| 1790.0|
|63b964e79dee32a35...|        6072|         2501664.0|  412.0|
|46bb3c0b1a65c8399...|         748|         2336752.0| 3124.0|
|05455dfa7cd02f13d...|        2184| 2160194.400000087|  989.1|
|3690e975641f01bd0...|         802|         2124498.0| 2649.0|
|349509b216bd5ec11...|         743|         1923627.0| 2589.0|
|695476b5848d64ba0...|         687|1820543.1299999943|2649.99|
|73236a0796f53d60d...|         832|         1755520.0| 2110.0|
|cc803a2c412833101...|         762|         1676400.0| 2200.0|
|1ff773612ab8934db...|        5820|1658641.7999999512| 

In [78]:
# Customer Segmentation based on spending

customer_spending_df = customer_spending_df.withColumn(
    'customer_segment',
    when(col('AOV') >= 1200, "High_Value")
    .when( (col('AOV')<1200) & (col('AOV')>=700),'Medium_Value')
          .otherwise('Low_Value')
         )

In [43]:
customer_spending_df.show()

+--------------------+------------+------------------+-------+----------------+
|         customer_id|total_orders|       total_spend|    AOV|customer_segment|
+--------------------+------------+------------------+-------+----------------+
|d3e82ccec3cb5f956...|        6876|         6662844.0|  969.0|    Medium_Value|
|df55c14d1476a9a34...|         743|         3565657.0| 4799.0|      High_Value|
|fe5113a38e3575c04...|        2292|         3293604.0| 1437.0|      High_Value|
|ec5b2ba62e5743423...|        1428|         2556120.0| 1790.0|      High_Value|
|63b964e79dee32a35...|        6072|         2501664.0|  412.0|       Low_Value|
|46bb3c0b1a65c8399...|         748|         2336752.0| 3124.0|      High_Value|
|05455dfa7cd02f13d...|        2184| 2160194.400000087|  989.1|    Medium_Value|
|3690e975641f01bd0...|         802|         2124498.0| 2649.0|      High_Value|
|349509b216bd5ec11...|         743|         1923627.0| 2589.0|      High_Value|
|695476b5848d64ba0...|         687|18205

In [79]:
full_order_df = full_order_df.join(customer_spending_df\
.select('customer_id','customer_segment'),'customer_id',how='left')

In [45]:
full_order_df.select('customer_id','customer_segment').show()

+--------------------+----------------+
|         customer_id|customer_segment|
+--------------------+----------------+
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
|3ce436f183e68e078...|       Low_Value|
+--------------------+----------------+
only showing top 20 rows



In [80]:
# Hourly Order Distribution

full_order_df = full_order_df.withColumn('hour_of_day',expr('hour(order_purchase_timestamp)'))

In [47]:
full_order_df.select('order_purchase_timestamp','hour_of_day').show()

+------------------------+-----------+
|order_purchase_timestamp|hour_of_day|
+------------------------+-----------+
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
+------------------------+-----------+
only showing top 20 rows



In [81]:
#weekday vs weekend Order

full_order_df = full_order_df.withColumn('day_of_week',dayofweek('order_purchase_timestamp'))

full_order_df.select('order_purchase_timestamp','day_of_week').show()

+------------------------+-----------+
|order_purchase_timestamp|day_of_week|
+------------------------+-----------+
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
|     2017-09-13 08:59:02|          4|
+------------------------+-----------+
only showing top 20 rows



In [82]:
full_order_df = full_order_df.withColumn('day_type',when(col('day_of_week')\
.isin(1,7),'weekend').otherwise('weekday'))

In [54]:
weekday_weekend_df = full_order_df.groupBy('day_type')\
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('total_revenue'),
    round(avg('price'),2).alias('avg_order_value')
).withColumn('total_revenue',format_number('total_revenue',2))


weekday_weekend_df.show()

+--------+------------+----------------+---------------+
|day_type|total_orders|   total_revenue|avg_order_value|
+--------+------------+----------------+---------------+
| weekday|    13965495|1,691,485,429.28|         121.12|
| weekend|     4098766|  501,338,749.25|         122.31|
+--------+------------+----------------+---------------+



In [55]:
# top category on weekday and weekend

top_category_df = full_order_df.groupBy('day_type','product_category_name')\
.agg(count('order_id').alias('total_orders'))\
.orderBy('day_type','total_orders',ascending=False)

top_category_df.show()

+--------+---------------------+------------+
|day_type|product_category_name|total_orders|
+--------+---------------------+------------+
| weekend|      cama_mesa_banho|      444149|
| weekend|     moveis_decoracao|      340348|
| weekend|         beleza_saude|      316514|
| weekend|        esporte_lazer|      309441|
| weekend| utilidades_domest...|      270263|
| weekend| informatica_acess...|      244683|
| weekend|   relogios_presentes|      204671|
| weekend|           brinquedos|      152681|
| weekend|            telefonia|      151418|
| weekend|   ferramentas_jardim|      151140|
| weekend|           cool_stuff|      139754|
| weekend|           perfumaria|      135180|
| weekend|           automotivo|      129791|
| weekend|                bebes|      114737|
| weekend|          eletronicos|      107330|
| weekend|            papelaria|       87044|
| weekend|             pet_shop|       86179|
| weekend| fashion_bolsas_e_...|       79782|
| weekend|    moveis_escritorio|  

In [83]:
# a new column frieght category based on freight value

full_order_df = full_order_df.withColumn('freight_category',
when(col('freight_value') >= 20, "High_Value")
    .when( (col('freight_value')<20) & (col('freight_value')>=13),'Medium_Value')
          .otherwise('Low_Value')
         )
                                         

In [61]:
full_order_df.select('freight_value','freight_category').show()

+-------------+----------------+
|freight_value|freight_category|
+-------------+----------------+
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
|        13.29|    Medium_Value|
+-------------+----------------+
only showing top 20 rows



In [57]:
# Order volume by customer state

Order_customer_state_df= full_order_df.groupBy('customer_state')\
.agg(count('order_id').alias('total_order')).orderBy(desc('total_order'))

Order_customer_state_df.show()



+--------------+-----------+
|customer_state|total_order|
+--------------+-----------+
|            SP|    6742255|
|            RJ|    3626875|
|            MG|    3433239|
|            RS|     971696|
|            PR|     746540|
|            SC|     644930|
|            BA|     443992|
|            ES|     367217|
|            GO|     162430|
|            MT|     155233|
|            PE|     132005|
|            DF|     109466|
|            PA|      96279|
|            CE|      74749|
|            MS|      73693|
|            MA|      61710|
|            AL|      37742|
|            PB|      33381|
|            SE|      28146|
|            PI|      27696|
+--------------+-----------+
only showing top 20 rows



In [84]:
full_order_df.write.mode('overwrite').parquet('/data/olist_proc')

In [85]:
spark.stop()